In [238]:
# Importar las bibliotecas necesarias
import numpy as np
import pandas as pd

In [239]:
#Extraer de ambos datasets modificados

movies= pd.read_csv('Dataset/movies_final.csv', encoding='utf-8')

credits_cast= pd.read_csv('Dataset/cast_credits.csv')
credits_crew= pd.read_csv('Dataset/crew_credits.csv')

In [240]:
print(credits_cast.shape)
credits_cast.head(1)

(562474, 9)


,cast_cast_id,cast_character,cast_credit_id,cast_gender,cast_id,cast_name,cast_order,cast_profile_path,id
0,14,Woody (voice),52fe4284c3a36847f8024f95,2,31,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,862.0


In [241]:
print(credits_crew.shape)
credits_crew.head(1)

(464314, 8)


,crew_credit_id,crew_department,crew_gender,crew_id,crew_job,crew_name,crew_profile_path,id
0,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,862.0


Se observan los dos datasets donde procedere a eliminar las colunas de los id menos el id final que me unira con el dataset de movies

In [242]:
# Eliminar las columnas que no serán utilizadas en cast
columns_to_drop = ['cast_cast_id', 'cast_credit_id','cast_gender', 'cast_id','cast_profile_path']
cast_df = credits_cast.drop(columns=columns_to_drop)

In [243]:
# Eliminar las columnas que no serán utilizadas en crew
columns_to_drop = ['crew_credit_id','crew_gender', 'crew_id', 'crew_profile_path']
crew_df = credits_crew.drop(columns=columns_to_drop)

In [ ]:
cast_df.head()

In [ ]:
crew_df.head()

In [244]:
#Renombrar los id
cast_df.rename(columns={'id': 'idMovies'}, inplace=True)
crew_df.rename(columns={'id': 'idMovies'}, inplace=True)

#Renombrar en nombre de actor
cast_df.rename(columns={'cast_name': 'nameActor'}, inplace=True)

Tabla cast

In [245]:
# Verificar valores nulos
print(cast_df.isnull().sum())

cast_character     25783
nameActor              0
cast_order             0
idMovies          516998
dtype: int64


In [246]:
#  Tratar valores nulos (eliminar)
cast_df= cast_df.dropna(subset=['idMovies'])  

# imputar valores nulos en otras columnas si es necesario
cast_df['cast_character'].fillna(value='cast_character', inplace=True)

C:\Users\evely\AppData\Local\Temp\ipykernel_15364\3264544608.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cast_df['cast_character'].fillna(value='cast_character', inplace=True)


In [247]:
# Verificar de nuevo 
print(cast_df.isnull().sum())

cast_character    0
nameActor         0
cast_order        0
idMovies          0
dtype: int64


AHora se verifican si existen duplicados

In [248]:

# Verificar duplicados basados en la columna 'idMovies' de cast
duplicates_cast_by_id = cast_df[cast_df.duplicated(subset=['idMovies'], keep=False)]

# Verificar duplicados en todas las columnas
duplicates_cast_all = cast_df[cast_df.duplicated(keep=False)]

# Mostrar duplicados
print("Duplicados basados en la columna 'idMovies' de la tabla cast:", len(duplicates_cast_by_id))
print("Duplicados en todas las columnas:", len(duplicates_cast_all))


Duplicados basados en la columna 'idMovies' de la tabla cast: 87
Duplicados en todas las columnas: 0


Se procede a eliminar todos los duplicados por el id de movies en la tabla cast

In [249]:

# Eliminar duplicados basados en 'idMovies', manteniendo solo la primera ocurrencia
cast_df = cast_df.drop_duplicates(subset=['idMovies'], keep='first')

# Verificar duplicados después de la eliminación
duplicates_after_cleaning = cast_df[cast_df.duplicated(subset=['idMovies'], keep=False)]
print("Duplicados basados en 'idMovies' después de la eliminación:", len(duplicates_after_cleaning))

Duplicados basados en 'idMovies' después de la eliminación: 0


Tabla Crew

In [250]:
# Verificar valores nulos
print(crew_df.isnull().sum())

crew_department         0
crew_job                0
crew_name               0
idMovies           418838
dtype: int64


In [251]:
#  Tratar valores nulos (eliminar) de id
crew_df= crew_df.dropna(subset=['idMovies'])  


Se eliminarion todos los nulos del campo id de la tabla crew

In [252]:
# Verificar de nuevo 
print(crew_df.isnull().sum())

crew_department    0
crew_job           0
crew_name          0
idMovies           0
dtype: int64


AHora se verifican si existen duplicados

In [253]:

# Verificar duplicados basados en la columna 'idMovies' de cast
duplicates_crew_by_id = crew_df[crew_df.duplicated(subset=['idMovies'], keep=False)]

# Verificar duplicados en todas las columnas
duplicates_crew_all = crew_df[crew_df.duplicated(keep=False)]

# Mostrar duplicados
print("Duplicados basados en la columna 'idMovies' de la tabla cast:", len(duplicates_crew_by_id))
print("Duplicados en todas las columnas:", len(duplicates_crew_all))

Duplicados basados en la columna 'idMovies' de la tabla cast: 87
Duplicados en todas las columnas: 0


No existen duplicados en la tabla crew

Se procede a unir ambas tablas con la de movies

In [254]:
print(crew_df.columns)

Index(['crew_department', 'crew_job', 'crew_name', 'idMovies'], dtype='object')


In [255]:
# Selecciona las columnas relevantes de cada DataFrame
cast_selected = cast_df[['cast_character', 'nameActor', 'cast_order', 'idMovies']]
crew_selected = crew_df[['crew_department', 'crew_job', 'crew_name', 'idMovies']]

# Realiza el merge
merged_df = pd.merge(cast_selected, crew_selected, on='idMovies', how='inner')

# Renombra columnas si es necesario
merged_df.rename(columns={
    'cast_character': 'character',  
    'crew_name': 'reparto_name'
}, inplace=True)

# Ver el resultado
print(merged_df.head())


                 character      nameActor  cast_order  idMovies  \
0            Woody (voice)      Tom Hanks           0     862.0   
1   Buzz Lightyear (voice)      Tim Allen           1    8844.0   
2  Mr. Potato Head (voice)    Don Rickles           2   15602.0   
3       Slinky Dog (voice)     Jim Varney           3   31357.0   
4              Rex (voice)  Wallace Shawn           4   11862.0   

  crew_department    crew_job    reparto_name  
0       Directing    Director   John Lasseter  
1         Writing  Screenplay     Joss Whedon  
2         Writing  Screenplay  Andrew Stanton  
3         Writing  Screenplay      Joel Cohen  
4         Writing  Screenplay    Alec Sokolow  


In [256]:
merged_df.head(1)

,character,nameActor,cast_order,idMovies,crew_department,crew_job,reparto_name
0,Woody (voice),Tom Hanks,0,862.0,Directing,Director,John Lasseter


In [257]:
# Unir el resultado anterior con basándose en 'idMovies'
merged_df = pd.merge(merged_df, movies, on='idMovies', how='inner')

In [258]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45451 entries, 0 to 45450
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   character             45451 non-null  object 
 1   nameActor             45451 non-null  object 
 2   cast_order            45451 non-null  int64  
 3   idMovies              45451 non-null  float64
 4   crew_department       45451 non-null  object 
 5   crew_job              45451 non-null  object 
 6   reparto_name          45451 non-null  object 
 7   budget                45451 non-null  int64  
 8   original_language     45451 non-null  object 
 9   overview              45451 non-null  object 
 10  popularity            45451 non-null  float64
 11  release_date          45451 non-null  object 
 12  revenue               45451 non-null  float64
 13  runtime               45451 non-null  float64
 14  status                45451 non-null  object 
 15  tagline            

In [259]:
# Verificar la cantidad de valores nulos por columna
null_counts = merged_df.isnull().sum()

# Mostrar las columnas con valores nulos y la cantidad de nulos en cada una
print(null_counts[null_counts > 0])


production_companies      1
production_countries      3
spoken_language         413
dtype: int64


In [260]:
# rellenar esa columnas 
fill_values = {
    'production_companies': 'unknown', 
    'production_countries': 'unknown',
    'spoken_language': 'unknown', 
}

# Rellenar los valores nulos en las columnas especificadas usando un diccionario
merged_df.fillna(value=fill_values, inplace=True)

In [261]:
# Verificar la cantidad de valores nulos por columna
merged_df.isnull().sum()

character               0
nameActor               0
cast_order              0
idMovies                0
crew_department         0
crew_job                0
reparto_name            0
budget                  0
original_language       0
overview                0
popularity              0
release_date            0
revenue                 0
runtime                 0
status                  0
tagline                 0
title                   0
vote_average            0
vote_count              0
genres                  0
production_companies    0
production_countries    0
spoken_language         0
release_year            0
return                  0
dtype: int64

In [263]:
#Verificar tamaño total de dataset
merged_df.shape

(45451, 25)

In [264]:
# Guardar el DataFrame unido en un archivo Parquet
merged_df.to_parquet('Dataset/Credits_movies_final.parquet', index=False)
